# Revised Hidden Weight Boolean Function

Automatic formatting:

In [1]:
%load_ext lab_black

Define variables:

In [2]:
z, p, q, r, u, v = var("z p q r u v")

For an even integer $n\geq 0$ and a Boolean vector $a=(a_i)_{i\in[1,n]}\in\mathbb F_2^n$, define:
- $p=p(a)$ to be the number of odd integers $i$ with $(a_i,a_{i+1})=(0,0)$.
- $q=q(a)$ to be the number of odd integers $i$ with $(a_i,a_{i+1})=(1,1)$.
- $r=p(a)$ to be the number of odd integers $i$ with $(a_i,a_{i+1})\in\{(0,1),(1,0)\}$.

It then holds that $p+q+r=n/2$. Consider then the polynomial $P_a(z)=(-z^2+2z+1)^p\cdot(-z^2-2z+1)^q\cdot(z^2+1)^r$. We can then write:
$$P_a(z)=\sum_{k=0}^n\mathsf D_{n,k,a}z^k,\qquad\mathsf D_{n,k,a}\in\mathbb Z.$$

In [3]:
P_a = (-z ^ 2 + 2 * z + 1) ^ p * (-z ^ 2 - 2 * z + 1) ^ q * (z ^ 2 + 1) ^ r

Through derivation, we can get $\mathsf D_{n,k,a}$ explicitly for small $k$:

In [4]:
k = 3
D_nka = diff(P_a, z, k)(z=0) / factorial(k)
print(f"D_(n, {k}, a) equals:\n    {D_nka.expand().factor()}.")

D_(n, 3, a) equals:
    2/3*(2*p^2 - 4*p*q + 2*q^2 - 9*p - 9*q + 3*r + 7)*(p - q).


It may be useful to express $\mathsf D_{n,k,a}$ not as a polynomial in $p, q, r$, but rather as a polynomial in $u,v,r$ with $u=p-q$ and $v=p+q$:

In [5]:
k = 3
D_nka = diff(P_a, z, k)(z=0) / factorial(k)
D_nka = D_nka.subs(p == (v + u) / 2, q=(v - u) / 2)
print(f"D_(n, {k}, a) equals:\n    {D_nka.expand().factor()}.")

D_(n, 3, a) equals:
    2/3*(2*u^2 + 3*r - 9*v + 7)*u.


For fixed $n$ and $k$, we would like to understand what the values for $(p,q,r)$ must be in order for $|\mathsf D_{n,k,a}|$ to be maximal:

In [6]:
n = 14
k = 9

tuples_pqr = []  # Collect the tuples (p, q, r)
abs_coeffs = []  # Collect the |D_(n, k, a)| for these tuples (p, q, r)

for p in range(n / 2 + 1):
    for q in range(n / 2 + 1):
        r = n / 2 - p - q
        if r < 0:
            continue
        tuples_pqr.append((p, q, r))
        abs_coeffs.append(abs(P_a(p=p, q=q, r=r).list()[k]))

max_value = max(abs_coeffs)
max_indices = [i for i, value in enumerate(abs_coeffs) if value == max_value]
print(f"The maximal value of |D_({n}, {k}, a)| is:\n    {max_value}")
print(
    f"This maximum is reached at the following values for (p, q, r):\n    {[tuples_pqr[i] for i in max_indices]}"
)

The maximal value of |D_(14, 9, a)| is:
    298
This maximum is reached at the following values for (p, q, r):
    [(1, 6, 0), (6, 1, 0)]
